In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('C:\\Users\\Aruna\\Documents\\ACMS-IID\\Code\\Data Cleaning\\Data1.csv')
df.head()

,label,description
0,Amazon S3,"['public', 'access', 'set', 'bucket', 'policy'..."
1,Amazon S3,"['figure', 'display', 'website', 'domain', 'Er..."
2,Amazon S3,"['test', 'ignore']"
3,Amazon S3,"['account', 'suspended', 'without', 'Notificat..."
4,Amazon S3,"['Unable', 'read', '1000', 'objects', 'bucket'..."


In [3]:
df['label'].value_counts()

Amazon S3                   53174
Amazon EC2                  46535
Amazon RDS                  29525
Amazon CloudFront           22314
Amazon Elastic Beanstalk    22246
Amazon VPC                  21809
Amazon Lambda                8439
Amazon SNS                   4530
Name: label, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split

X = df['description']  # this time we want to look at the text
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(139743, 103455)

In [6]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [7]:
from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('rfc',RandomForestClassifier()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

C:\Users\Aruna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [8]:
predictions = text_clf.predict(X_test)

In [9]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[ 4509   455   115    46   232  2079     6   109]
 [  406 11362   516    76   888  1507    16   634]
 [  207  1133  4715    51   344   859     8   126]
 [  140   358   207  1111   182   666    15    81]
 [  234  1647   270    53  6154  1003    15   273]
 [ 1033  1323   347   130   588 13767    17   212]
 [   74   160    73    46    96   466   539    19]
 [  131  1576   150    31   388   591     4  4260]]


In [10]:
print(metrics.classification_report(y_test,predictions))

                          precision    recall  f1-score   support

       Amazon CloudFront       0.67      0.60      0.63      7551
              Amazon EC2       0.63      0.74      0.68     15405
Amazon Elastic Beanstalk       0.74      0.63      0.68      7443
           Amazon Lambda       0.72      0.40      0.52      2760
              Amazon RDS       0.69      0.64      0.66      9649
               Amazon S3       0.66      0.79      0.72     17417
              Amazon SNS       0.87      0.37      0.52      1473
              Amazon VPC       0.75      0.60      0.66      7131

               micro avg       0.67      0.67      0.67     68829
               macro avg       0.72      0.60      0.63     68829
            weighted avg       0.68      0.67      0.67     68829



In [11]:
print(metrics.accuracy_score(y_test,predictions))

0.6743814380566331
